In [103]:
%load_ext autoreload
%autoreload 2
import utils; utils.setup()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
import numpy as np

import evaluation
import data

In [105]:
from pathlib import Path
project_root = Path().resolve().parent

intervals = data.intervals


In [106]:
raw_interval = '15m'
interval = '1h'

raw_data_1min = data.load_symbol('solusdt', dir=project_root, interval=raw_interval)


In [109]:
df = data.set_interval(raw_data_1min, intervals[interval])

In [108]:
gain_threshold = 0.02
periods = 8

def indicator(df, date: np.datetime64):
    hit = True
    return evaluation.Prediction(date=date, state='valid', hit=hit)

evaluation.confusion_matrix(df, gain_threshold=gain_threshold, periods=periods, indicator=indicator)

TP    1508
FP       0
FN    5262
TN       0
dtype: int64